### Installation

In [ ]:
!pip install pymupdf
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!sudo apt-get install unzip
!unzip -q "/content/gdrive/My Drive/CUAD_v1.zip"

Mounted at /content/gdrive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from operator import itemgetter
import fitz
import json

In [ ]:
import re 
from unidecode import unidecode 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_empty(df):
  df = df.reset_index(drop=True)

  for i in range(len(df)-1):
      if df.at[i, 'content'] == '' and df.at[i, 'heading'] != '':
          df.at[i+1, 'heading'] = df.at[i, 'heading'] + '. ' + df.at[i+1, 'heading']
          df.drop(i, inplace=True)
    
      elif df.at[i, 'heading'] == '' and df.at[i, 'content'] != '':
          df.at[i+1, 'content'] = df.at[i, 'content'] + '. ' + df.at[i+1, 'content']
          df.drop(i, inplace=True)

  return df

In [ ]:
def split_section_num(df):
# Define a regex pattern to detect section numbers
  section_pattern = re.compile(r'^\s*(\d*\.\s+|\d*\.|M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.\s+|M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\.)+')

  # Define an empty list to store the results
  result = []

  # Iterate over the rows of the dataframe
  for _, row in df.iterrows():
    
      # Split the content column into individual sentences
      sentences = row['content'].split('\n')

      # Define a variable to store the heading
      heading = None
      
      # Iterate over the sentences
      for sentence in sentences:
          # Check if the sentence is a section heading
          if section_pattern.match(sentence):
              # If it is, set the heading variable to the first sentence
              heading = sentence
              
              # Append the section heading as a new row to the result list
              result.append({
                  'heading': heading,
                  'content': ''
              })
          elif heading:
              # If there is a heading, append the sentence to the content of the previous section
              result[-1]['content'] += sentence + '\n'
          
          else:
              # If there is no heading, keep the original heading and add the sentence to the content
              if any(r['heading'] == row['heading'] for r in result):
                  # If a row with the same heading already exists, append the content to its existing content
                  result[-1]['content'] += '\n' + sentence
              else:
                  # Otherwise, create a new row with the original heading and the sentence as the content
                  result.append({
                      'heading': row['heading'],
                      'content': sentence
                  })

  # Convert the result list to a new dataframe
  new_df = pd.DataFrame(result)

  return new_df

In [ ]:
def extract_segement(doc):
  # get the spans from the PDF file
  block_dict = {}

  page_num = 1
  for page in doc: # Iterate all pages in the document
        file_dict = page.get_text('dict') # Get the page dictionary 
        block = file_dict['blocks'] # Get the block information
        block_dict[page_num] = block # Store in block dictionary
        page_num += 1 # Increase the page value by 1

  # create a dictionary containing detailed information of all spans in the document and store in a DataFrame
  spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
  rows = []

  for page_num, blocks in block_dict.items():
      for block in blocks:
          if block['type'] == 0:
              for line in block['lines']:
                  for span in line['spans']:
                      xmin, ymin, xmax, ymax = list(span['bbox'])
                      font_size = span['size']
                      text = unidecode(span['text'])
                      span_font = span['font']
                      
                      is_upper = False
                      is_bold = False                     

                      if "bold" in span_font.lower():
                          is_bold = True 
                      if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                          is_upper = True                 
                      if text.replace(" ","") !=  "":
                          rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))
                          
  span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])

  # create more features
  """
  The ‘h’ tag denotes the text which is bigger and more important than normal paragraphs. (e.g. UPPER CASE and bold style)
  The ‘p’ tag stands for paragraph, or the normal content in the document. 
  The ‘s’ tag will be used for less important text, which is smaller than ‘p’ text.
  """

  span_scores = []
  span_num_occur = {}
  special = '[(_:/,#%\=@)]'

  for index, span_row in span_df.iterrows():    
      score = round(span_row.font_size)
      text = span_row.text    
      if not re.search(special, text):       
          if span_row.is_bold:
              score +=1 
          if span_row.is_upper:
              score +=1
      span_scores.append(score)

  values, counts = np.unique(span_scores, return_counts=True)
  style_dict = {}

  for value, count in zip(values, counts):
      style_dict[value] = count

  sorted(style_dict.items(), key=lambda x: x[1])

  p_size = max(style_dict, key=style_dict.get)
  idx = 0
  tag = {}

  for size in sorted(values, reverse = True):
      idx += 1
      if size == p_size:
          idx = 0
          tag[size] = 'p'
      if size > p_size:
          tag[size] = 'h{0}'.format(idx)
      if size < p_size:
          tag[size] = 's{0}'.format(idx)

  span_tags = [tag[score] for score in span_scores]
  span_df['tag'] = span_tags

  # define headings and paragraphs in the document
  headings_list = []
  text_list = []
  tmp = []
  heading = ''                                                                                                                

  for index, span_row in span_df.iterrows():
      text = span_row.text
      tag = span_row.tag
      if 'h' in tag:
        if len(text) < 70:
          headings_list.append(text)
          text_list.append('\n'.join(tmp)) 
          tmp = []
          heading = text
        else:
          tmp.append(text) 
      else:
          tmp.append(text)

  text_list.append('\n'.join(tmp))
  text_list = text_list[1:]
  text_df = pd.DataFrame(zip(headings_list, text_list),columns=['heading', 'content'] )

  # combine the empty cells
  text_df = remove_empty(text_df)

  # split the section number out
  new_df = split_section_num(text_df)

  # combine the empty cells
  new_df = remove_empty(new_df)

  return(list(new_df['heading'] + " " + new_df['content']))

In [ ]:
# load the file
my_path = '/content/CUAD_v1/full_contract_pdf/Part_I/Promotion/DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_11414857_EX-10.2_Promotion Agreement.pdf'
doc = fitz.open(my_path)
my_path2 = '/content/CUAD_v1/full_contract_pdf/Part_I/Promotion/ExactSciencesCorp_20180822_8-K_EX-10.1_11331629_EX-10.1_Promotion Agreement.pdf'
doc2 = fitz.open(my_path2)

In [ ]:
doc_1 = extract_segement(doc)
doc_2 = extract_segement(doc2)

                                               heading  \
0               CO-PROMOTION AGREEMENT. by and between   
1       and. VALEANT PHARMACEUTICALS NORTH AMERICA LLC   
2       TABLE OF CONTENTS. Page. ARTICLE 1 DEFINITIONS   
3                     ARTICLE 2 RIGHTS AND OBLIGATIONS   
4    2.1.  Engagement? Grant of Rights.. 8. 2.2.  R...   
..                                                 ...   
479                                About Bausch Health   
480          Bausch Health Forward-looking Statements    
481                                     PM-US-DOP-0072   
482                                    SAL.0103.USA.18   
484  1. Stock Purchase Agreement dated March 29, 20...   

                                               content  
0                           DOVA PHARMACEUTICALS, INC.  
1    September 26, 2018\n__________________________...  
2                                                    1  
3                                                    8  
4                 

In [ ]:
# Use nltk's English stopwords
stopwords = nltk.corpus.stopwords.words('english') 
new_stopwords = ['shall','agreement','contract','party','date','one','may','hundred','time','will','make','term','section','thousand']
stopwords.extend('new_stopwords')

In [ ]:
def similarity_para(playbook, document):

  # exclude stop words and tokenize words of paragraphs (NLTK) and create dictionary of the playbook
  gen_docs = [[w.lower() for w in word_tokenize(i) if w not in stopwords] for i in playbook]
  dictionary = gensim.corpora.Dictionary(gen_docs)
  # create a bag of words and its frequency in each document
  corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
  # calculate TFIDF
  tf_idf = gensim.models.TfidfModel(corpus)
  # creating similarity measure object
  sims = gensim.similarities.Similarity('/content/Untitled Folder',tf_idf[corpus], num_features=len(dictionary))

  # processing paragraphs of the second document
  for i in range(len(document)):
    query_doc = [w.lower() for w in word_tokenize(document[i]) if w not in stopwords]
    query_doc_bow = dictionary.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # find similarity for each document
    score = sims[query_doc_tf_idf]
    # print(score)
    print("Doc2 ", document[i])
    # match the paragaphs with the highest score
    if np.max(score) != 0:
      indices = np.argmax(score)
      print("Doc1 ", playbook[indices])
    else:
      print("No Section Matched")
    print()

In [ ]:
similarity_para(doc_1, doc_2)

Doc2  Exhibit 10.1 COLOGUARD(r)
Doc1  10.1.  Representations and Warranties of Dova.. 35. 10.2.  Representations and Warranties of Valeant.. 37. 10.3.  Disclaimer of Warranty.. 38. 10.4.  Additional Covenants.. 39. ARTICLE 11 INDEMNIFICATION? LIMITATIONS ON LIABILITY 39

Doc2  PROMOTION AGREEMENT. BY AND BETWEEN. EXACT SCIENCES CORPORATION. AND. PFIZER INC. August 21, 2018
Source: EXACT SCIENCES CORP, 8-K, 8/22/2018
Doc1  Effective. Date ") by and between Dova Pharmaceuticals, Inc., a Delaware corporation ("

Doc2  TABLE OF CONTENTS Page
Doc1  TABLE OF CONTENTS. Page. ARTICLE 1 DEFINITIONS 1

Doc2  2.1 Joint Steering Committee
10

Doc1  2.1.  Engagement? Grant of Rights.. 8. 2.2.  Retention of Rights.. 9. 2.3.  Non-Competition? Non-Solicitation.. 9. 2.4.  Dova Trademarks and Copyrights.. 10. ARTICLE 3 JOINT STEERING COMMITTEE 11

Doc2  2.2 Joint Operations Committee
13

Doc1  2.1.  Engagement? Grant of Rights.. 8. 2.2.  Retention of Rights.. 9. 2.3.  Non-Competition? Non-Solicitation..